**Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho

Grupo ( A )

---


Aluna(o): Luana Guedes Barros Martins

---

Aluna(o): Ruan Chaves Rodrigues

---

Aluna(o): Thiago Monteles de Souza

---

# Atividade Prática 04 - Sistemas Nebulosos


In [1]:
# Imports

import numpy as np
import pandas as pd
from functools import reduce
import seaborn as sns
import re

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
!pip install -U scikit-fuzzy

Requirement already up-to-date: scikit-fuzzy in /home/luana/miniconda3/envs/IA/lib/python3.7/site-packages (0.4.1)


In [3]:
import skfuzzy as fuzz

## Tratamento de dados

A partir da base de dados dos alunos do curso de Ciência da Computação disponibilizada, deve-se projetar e implementar  um sistema nebuloso para classificar alunos do curso de Ciência da Computação. A classificação deverá ser feita incrementalmente do primeiro ao último período, e os resultados mostrados em um gráfico. Deve ser possível observar o desempenho de um aluno específico e também de todos os alunos em conjunto.

Para isso, defina as variáveis nebulosas descritas a seguir. As variáveis devem ter 3 faixas de valores.

**Entrada**:

- Média em disciplinas de programação
- Média em disciplinas básicas de matemática
- Média em disciplinas do núcleo específico
- Média em disciplinas do núcleo comum
- Quantidade de reprovações  em disciplinas de programação
- Quantidade de reprovações em disciplinas básicas de matemática
- Quantidade de reprovações em disciplinas do núcleo específico
- Quantidade de reprovações em disciplinas do núcleo comum
- Quantidade de trancamentos

**Saída**:

    Classificação do aluno.

Para que seja atendido o requisito de observar o desempenho de um aluno de maneira incremental ao longo dos períodos, comparando-o com outros alunos, foi estabelecido um novo conceito para período: iremos tratar como período uma faixa de valores de integralização do curso. De 0% a 10% de integralização temos o período 0, de 10% a 20% temos o segundo período e assim por diante.

In [4]:
dataset = pd.read_csv('baseDeDados.csv', sep=';')

/home/luana/miniconda3/envs/IA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def tabela_nucleo(info, matriz):
    info = info.split('\n')[1:]
    componentes = pd.DataFrame([x.split('\t') for x in info])\
    .drop(columns=[1,2,4])\
    .dropna()
    componentes = componentes.rename(columns={
        0: "codigo_componente",
        3: "nucleo"
    })
    componentes['codigo_matriz_curricular'] = matriz
    return componentes

def tabela_integralizacao(dataset, matriz):
    tmp = dataset[dataset['codigo_matriz_curricular'] == matriz]\
    [['id','ano_periodo_cursado_componente', 'nome_componente', 'nota_discente_componente']]

    tmp['nota_discente_componente'] = tmp['nota_discente_componente'].apply(lambda x: 1 if x >= 6.0 else 0)
    tmp = tmp\
    .groupby(by=['id', 'ano_periodo_cursado_componente'])\
    .agg([np.sum])\
    .groupby('id')\
    .apply(lambda x: x.cumsum())\
    .reset_index()

    tmp['nota_discente_componente'] = tmp['nota_discente_componente'] / tmp['nota_discente_componente'].max()
    tmp = tmp[['id', 'ano_periodo_cursado_componente', 'nota_discente_componente']]\
    .rename(columns={
        'nota_discente_componente': 'integralizacao'
    })
    tmp.columns = tmp.columns.get_level_values(0)
    return tmp

In [6]:
# Dados da antiga matriz curricular.

info1 = """
IME0073	CÁLCULO 1 - 64h	DISCIPLINA	NC	OBRIG.
INF0109	INTRODUÇÃO À COMPUTAÇÃO - 64h	DISCIPLINA	NC	OBRIG.
INF0131	LÓGICA MATEMÁTICA - 64h	DISCIPLINA	NC	OBRIG.
INF0135	MATEMÁTICA DISCRETA - 64h	DISCIPLINA	NC	OBRIG.
INF0156	PROGRAMAÇÃO DE COMPUTADORES 1 CN - 64h	DISCIPLINA	NC	OBRIG.
IFI0105	FÍSICA PARA COMPUTAÇÃO - 64h	DISCIPLINA	NC	OBRIG.
IME0006	ÁLGEBRA LINEAR - 64h	DISCIPLINA	NC	OBRIG.
IME0078	CÁLCULO 2 - 64h	DISCIPLINA	NC	OBRIG.
INF0157	PROGRAMAÇÃO DE COMPUTADORES 2 CN - 64h	DISCIPLINA	NC	OBRIG.
INF0195	SISTEMAS DIGITAIS - 64h	DISCIPLINA	NC	OBRIG.
IME0232	PROBABILIDADE E ESTATÍSTICA - 64h	DISCIPLINA	NC	OBRIG.
INF0015	ARQUITETURA DE COMPUTADORES - 64h	DISCIPLINA	NC	OBRIG.
INF0063	ESTRUTURAS DE DADOS I - 64h	DISCIPLINA	NC	OBRIG.
INF0164	PROGRAMAÇÃO ORIENTADA A OBJETOS - 64h	DISCIPLINA	NC	OBRIG.
INF0210	TEORIA DOS GRAFOS - 64h	DISCIPLINA	NC	OBRIG.
INF0013	ANÁLISE E PROJETO DE ALGORITMOS - 64h	DISCIPLINA	NC	OBRIG.
INF0022	BANCO DE DADOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0064	ESTRUTURAS DE DADOS II - 64h	DISCIPLINA	NC	OBRIG.
INF0124	LINGUAGENS DE PROGRAMAÇÃO - 64h	DISCIPLINA	NC	OBRIG.
INF0148	PESQUISA OPERACIONAL - 64h	DISCIPLINA	NC	OBRIG.
INF0055	ENGENHARIA DE SOFTWARE - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0090	INTELIGÊNCIA ARTIFICIAL - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0182	REDES DE COMPUTADORES 1 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0201	SISTEMAS GERENCIADORES DE BD - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0204	SISTEMAS OPERACIONAIS 1 - 64h	DISCIPLINA	NC	OBRIG.
INF0052	ENGENHARIA DE REQUISITOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0127	LINGUAGENS FORMAIS E AUTÔMATOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0185	REDES DE COMPUTADORES 2 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0196	SISTEMAS DISTRIBUÍDOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0205	SISTEMAS OPERACIONAIS 2 - 64h	DISCIPLINA	NC	OBRIG.
INF0031	COMPILADORES - 64h	DISCIPLINA	NC	OBRIG.
INF0037	COMPUTAÇÃO GRÁFICA - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0171	PROJETO DE SOFTWARE - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0174	PROJETO FINAL DE CURSO 1 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0209	TEORIA DA COMPUTAÇÃO - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0220	TÓPICOS 1 - 64h	DISCIPLINA	NE-Obr	OBRIG.
FAD0013	DIREITO - 64h	DISCIPLINA	NC	OBRIG.
INF0039	COMPUTADOR E SOCIEDADE - 32h	DISCIPLINA	NC	OBRIG.
INF0050	EMPREENDEDORISMO - 64h	DISCIPLINA	NC	OBRIG.
INF0143	MULTIMÍDIA - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0177	PROJETO FINAL DE CURSO 2 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0191	SEGURANÇA E AUDITORIA DE SISTEMAS - 32h	DISCIPLINA	NC	OBRIG.
INF0222	TÓPICOS 2 - 64h	DISCIPLINA	NE-Obr	OBRIG.
"""

# Dados da nova matriz curricular.
info2 = """
IME0075	CÁLCULO 1A - 96h	DISCIPLINA	NC	OBRIG.
INF0283	COMPUTAÇÃO E SOCIEDADE - 32h	DISCIPLINA	NC	OBRIG.
INF0284	INTRODUÇÃO À PROGRAMAÇÃO - 128h	DISCIPLINA	NC	OBRIG.
INF0285	FUNDAMENTOS DE MATEMÁTICA PARA COMPUTAÇÃO - 64h	DISCIPLINA	NC	OBRIG.
IFI0080	FÍSICA I - 64h	DISCIPLINA	NE-Obr	OBRIG.
IME0080	CÁLCULO 2A - 96h	DISCIPLINA	NE-Obr	OBRIG.
IME0164	GEOMETRIA ANALÍTICA - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0286	ALGORITMOS E ESTRUTURAS DE DADOS 1 - 64h	DISCIPLINA	NC	OBRIG.
INF0305	LÓGICA MATEMÁTICA - 64h	DISCIPLINA	NC	OBRIG.
IFI0091	FÍSICA III - 64h	DISCIPLINA	NE-Obr	OBRIG.
IME0006	ÁLGEBRA LINEAR - 64h	DISCIPLINA	NC	OBRIG.
IME0293	PROBABILIDADE E ESTATÍSTICA A - 64h	DISCIPLINA	NC	OBRIG.
INF0287	ALGORITMOS E ESTRUTURAS DE DADOS 2 - 64h	DISCIPLINA	NC	OBRIG.
INF0330	METODOLOGIA DE PESQUISA EM COMPUTAÇÃO - 32h	DISCIPLINA	NE-Obr	OBRIG.
INF0331	MATEMÁTICA DISCRETA - 64h	DISCIPLINA	NE-Obr	OBRIG.
EMC0292	CIRCUITOS DIGITAIS - 64h	DISCIPLINA	NE-Obr	OBRIG.
IME0108	EQUAÇÕES DIFERENCIAIS ORDINÁRIAS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0164	PROGRAMAÇÃO ORIENTADA A OBJETOS - 64h	DISCIPLINA	NC	OBRIG.
INF0313	PESQUISA OPERACIONAL - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0332	TEORIA DOS GRAFOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
EMC0152	ELETRÔNICA PARA COMPUTAÇÃO - 96h	DISCIPLINA	NE-Obr	OBRIG.
IME0065	CÁLCULO NUMÉRICO - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0013	ANÁLISE E PROJETO DE ALGORITMOS - 64h	DISCIPLINA	NC	OBRIG.
INF0015	ARQUITETURA DE COMPUTADORES - 64h	DISCIPLINA	NC	OBRIG.
INF0334	SOFTWARE BÁSICO - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0022	BANCO DE DADOS - 64h	DISCIPLINA	NC	OBRIG.
INF0056	ENGENHARIA DE SOFTWARE - 64h	DISCIPLINA	NC	OBRIG.
INF0288	LINGUAGENS E PARADIGMAS DE PROGRAMAÇÃO - 64h	DISCIPLINA	NC	OBRIG.
INF0333	LINGUAGENS FORMAIS E AUTÔMATOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0336	SISTEMAS OPERACIONAIS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0335	INTRODUÇÃO À COMPUTABILIDADE E À COMPLEXIDADE COMPUTACIONAL - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0339	ENGENHARIA DE REQUISITOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0340	SISTEMAS GERENCIADORES DE BANCO DE DADOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0341	REDES DE COMPUTADORES 1 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0174	PROJETO FINAL DE CURSO 1 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0290	PROJETO DE SOFTWARE - 64h	DISCIPLINA	NC	OBRIG.
INF0338	COMPUTAÇÃO GRÁFICA - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0342	INTELIGÊNCIA ARTIFICIAL - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0344	SISTEMAS DISTRIBUÍDOS - 64h	DISCIPLINA	NE-Obr	OBRIG.
FAL0214	INTRODUÇÃO À LÍNGUA BRASILEIRA DE SINAIS - LIBRAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0294	ENGENHARIA DE SISTEMAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0321	SEGURANÇA E AUDITORIA DE SISTEMAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0322	DESENVOLVIMENTO DE SISTEMAS PARA WEB - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0327	EMPREENDEDORISMO DIGITAL - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0328	COMPUTAÇÃO MÓVEL E UBÍQUA - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0343	REDES DE COMPUTADORES 2 - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0354	APRENDIZAGEM DE MÁQUINA E MINERAÇÃO DE DADOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0355	ARMAZÉM DE DADOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0356	AVALIAÇÃO DE DESEMPENHO DE SISTEMAS COMPUTACIONAIS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0357	BANCO DE DADOS DISTRIBUÍDOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0358	COMPUTAÇÃO EVOLUCIONÁRIA - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0359	COMPUTAÇÃO VERDE - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0360	DESENVOLVIMENTO DE SOFTWARE DIRIGIDO POR MODELOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0361	ENGENHARIA DE SOFTWARE BASEADA EM BUSCA - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0362	FUNDAMENTOS DA PROGRAMAÇÃO LÓGICA E FUNCIONAL - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0363	INTEGRAÇÃO DE DADOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0364	INTRODUÇÃO À TEORIA DAS PROVAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0365	INTRODUÇÃO À TEORIA DOS TIPOS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0366	JOGOS DIGITAIS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0367	META-HEURÍSTICAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0368	METODOLOGIA E EXPERIMENTAÇÃO EM ENGENHARIA DE SOFTWARE - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0369	MICROCONTROLADORES - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0370	PROCESSAMENTO DIGITAL DE IMAGENS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0371	RECUPERAÇÃO DE INFORMAÇÃO - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0372	REDES NEURONAIS ARTIFICIAIS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0373	SISTEMAS MULTIAGENTES - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0374	TÓPICOS AVANÇADOS EM REDES SEM FIO - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0375	TÓPICOS EM COMPUTAÇÃO - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0376	TÓPICOS EM TESTE DE SOFTWARE - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0377	VERIFICAÇÃO FORMAL DE PROGRAMAS - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0378	VISÃO ROBÓTICA - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0379	VISUALIZAÇÃO DE INFORMAÇÕES - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0380	WEB SEMÂNTICA - 64h	DISCIPLINA	NE-Opt	OPTAT.
INF0177	PROJETO FINAL DE CURSO 2 - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0289	INTERAÇÃO HUMANO-COMPUTADOR - 64h	DISCIPLINA	NC	OBRIG.
INF0337	COMPILADORES - 64h	DISCIPLINA	NE-Obr	OBRIG.
INF0345	COMPUTAÇÃO PARALELA - 64h	DISCIPLINA	NE-Obr	OBRIG.
"""

Vamos tratar o dataset adicionando uma coluna "núcleo" que indica NC para núcleo comum e NE para núcleo específico.

In [7]:
tabela1 = tabela_nucleo(info1, 'CICOMP-BI-2')
tabela2 = tabela_nucleo(info2, 'CICOMP-BI-3')
tabela_final = tabela1.append(tabela2)
dataset = pd.merge(dataset, tabela_final, how='inner', on=['codigo_componente', 
                                                           'codigo_matriz_curricular'])

In [8]:
tabela1 = tabela_integralizacao(dataset, 'CICOMP-BI-2')
tabela2 = tabela_integralizacao(dataset, 'CICOMP-BI-3')
tabela_final = tabela1.append(tabela2)
dataset = pd.merge(dataset, tabela_final, how='inner', on=['id', 
                                                           'ano_periodo_cursado_componente'])

In [9]:
# 

dataset['integralizacao'] = dataset['integralizacao'].apply(lambda x: int(x * 10) )
dataset.sample(5)

,id,ano_nascimento_discente,idade_conclusao_ensino_medio,idade_ingresso_universidade,idade_colacao_grau,uf_naturalidade_discente,discente_estudou_escola_publica?,ano_conclusao_ensino_medio,nome_municipio_curso,nome_campus_curso,...,frequencia_discente_componente,nota_discente_componente,situacao_discente_componente,quantidade_trancamentos,quantidade_reprovacoes_nc,quantidade_reprovacoes_nl,quantidade_reprovacoes_neobr,quantidade_reprovacoes_neopt,nucleo,integralizacao
22303,693,1996,NaN,20,NaN,GO,SIM,NaN,GOIÂNIA,SAMAMBAIA,...,0.88,7.7,AP,NaN,8.0,2.0,4.0,NaN,NC,0
4087,524,1995,17.0,19,NaN,GO,NÃO,2012.0,GOIÂNIA,SAMAMBAIA,...,1.00,7.1,AP,NaN,8.0,1.0,NaN,NaN,NC,3
2598,496,1996,17.0,18,NaN,GO,NÃO,2013.0,GOIÂNIA,SAMAMBAIA,...,0.97,7.3,AP,NaN,11.0,NaN,2.0,NaN,NC,0
17004,185,1993,16.0,17,24.0,GO,NÃO,2009.0,GOIÂNIA,SAMAMBAIA,...,0.78,5.4,AP,1.0,14.0,1.0,7.0,NaN,NC,5
9492,47,1988,18.0,20,27.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,0.81,3.5,RM,1.0,18.0,1.0,12.0,NaN,NC,3


In [10]:
# Função que calcula a média por período para cada aluno para um determinado
# tipo de disciplina
def media(coluna, chave, nome):
    tmp = dataset[dataset[coluna].str.contains(chave)]\
    [['id', 'nome_componente', 'nota_discente_componente', 'integralizacao']]\
    .groupby(by=['id', 'integralizacao'])\
    .agg([np.mean])\
    .reset_index()
    
    tmp.columns = tmp.columns.get_level_values(0)
    return tmp.rename(columns={
        "nota_discente_componente" : nome
    })

# Função que calcula a quantidade de reprovações por período para cada aluno
# para um determinado tipo de disciplina
def reprovacoes(coluna, chave, nome):
    tmp = dataset[dataset[coluna].str.contains(chave)]\
    [['id', 'nome_componente', 'nota_discente_componente', 'integralizacao']]

    tmp['nota_discente_componente'] = tmp['nota_discente_componente']\
    .apply(lambda x: 0 if x >= 6.0 else 1)

    tmp = tmp\
    .groupby(by=['id', 'integralizacao'])\
    .agg([np.sum])\
    .reset_index()\
    .drop(columns=['nome_componente'])\
    .rename(columns={
        "nota_discente_componente" : nome
    })
    tmp.columns = tmp.columns.get_level_values(0)
    return tmp

In [11]:
dataframes = [ 
# Média em disciplinas de programação
media('codigo_componente', 'INF', 'media_programacao'),
    
# Média em disciplinas básicas de matemática
media('codigo_componente', 'IME', 'media_matematica'),
    
# Média em disciplinas do núcleo específico
media('nucleo', 'NE', 'media_NE'),    
    
# Média em disciplinas do núcleo comum
media('nucleo', 'NC', 'media_NC'),
    
# Quantidade de reprovações em disciplinas de programação
reprovacoes('codigo_componente', 'INF', 'reprovacoes_programacao'),
    
# Quantidade de reprovações em disciplinas básicas de matemática
reprovacoes('codigo_componente', 'IME', 'reprovacoes_matematica'),
    
# Quantidade de reprovações em disciplinas do núcleo específico
reprovacoes('nucleo', 'NE', 'reprovacoes_NE'),    
    
# Quantidade de reprovações em disciplinas do núcleo comum
reprovacoes('nucleo', 'NC', 'reprovacoes_NC'),    
]

df = \
reduce(lambda x,y: pd.merge(x,
                            y,
                            how='outer', 
                            on=['id', 'integralizacao']), dataframes)

# Quantidade de trancamentos
trancamentos = \
dataset[['id', 'quantidade_trancamentos']].drop_duplicates().fillna(0.0).astype(int)

df = pd.merge(df, trancamentos, how='outer', on='id')

/home/luana/miniconda3/envs/IA/lib/python3.7/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [12]:
df.sort_values(by=['id', 'integralizacao']).head()

,id,integralizacao,media_programacao,media_matematica,media_NE,media_NC,reprovacoes_programacao,reprovacoes_matematica,reprovacoes_NE,reprovacoes_NC,quantidade_trancamentos
0,1,0,6.550000,7.3,NaN,6.700000,1.0,0.0,NaN,1.0,1
1,1,1,5.400000,5.2,3.800000,5.500000,6.0,1.0,1.0,7.0,1
2,1,2,4.956250,4.2,4.709091,5.283333,12.0,1.0,9.0,4.0,1
3,1,3,4.822222,NaN,4.466667,6.028571,13.0,NaN,10.0,3.0,1
4,1,4,7.066667,6.9,7.066667,6.900000,0.0,0.0,0.0,0.0,1


In [13]:
# Salva o dataset ordenado pelo id(aluno) e pela sua integralização
df.sort_values(by=['id', 'integralizacao']).to_csv('fuzzy_dataset.csv')

In [14]:
# Carregando o dataset
dataset = pd.read_csv('fuzzy_dataset.csv')
dataset.head()

,Unnamed: 0,id,integralizacao,media_programacao,media_matematica,media_NE,media_NC,reprovacoes_programacao,reprovacoes_matematica,reprovacoes_NE,reprovacoes_NC,quantidade_trancamentos
0,0,1,0,6.550000,7.3,NaN,6.700000,1.0,0.0,NaN,1.0,1
1,1,1,1,5.400000,5.2,3.800000,5.500000,6.0,1.0,1.0,7.0,1
2,2,1,2,4.956250,4.2,4.709091,5.283333,12.0,1.0,9.0,4.0,1
3,3,1,3,4.822222,NaN,4.466667,6.028571,13.0,NaN,10.0,3.0,1
4,4,1,4,7.066667,6.9,7.066667,6.900000,0.0,0.0,0.0,0.0,1


In [15]:
def transform_value(value):
    try:
        return int(str(value).rstrip(','))
    except:
        return str(value).rstrip(',')
    
medidas = df\
.groupby(by=['integralizacao'])\
.agg([np.min, np.mean, np.max])\
.transpose()\
.reset_index()\
.applymap(str)\
.apply(lambda x: x + ',')\
.groupby('level_0')\
.apply(lambda x: x.sum())\
.rename(columns=lambda x: transform_value(x))

medidas['level_0'] = medidas['level_0'].apply(lambda x: x.split(',')[0] )
drop = ['id', 'Unnamed: 0']
medidas = medidas[~medidas['level_0'].isin(drop)]


valores = df\
.applymap(str)\
.apply(lambda x: x + ',')\
.groupby(by=['integralizacao'])\
.agg([np.sum])\
.transpose()\
.reset_index()\
.rename(columns=lambda x: transform_value(x))

valores = valores[~valores['level_0'].isin(drop)]

In [16]:
def gen_dict(table):
    table.index = table['level_0']
    table_dict = table\
    .iloc[:, 2:]\
    .to_dict()
    
    for key in table_dict.keys():
        inner_dict = table_dict[key]
        for key2 in inner_dict.keys():
            inner_dict[key2] = inner_dict[key2].split(',')
            inner_dict[key2] = [ x for x in inner_dict[key2] if re.match("^(?=.)([+-]?([0-9]*)(\.([0-9]+))?)$", x) is not None ]
            inner_dict[key2] = [ round(float(x), 2) for x in inner_dict[key2] ]
            inner_dict[key2] = sorted(list(set(inner_dict[key2])))
            
    return table_dict

In [17]:
def flatten_dict(df):
    tmp = gen_dict(df)
    rows = []
    for key in sorted(tmp.keys()):
        for key2 in sorted(tmp[key].keys()):
            for value in tmp[key][key2]:
                row = {
                    "integralizacao": key,
                    "campo": key2,
                    "valor": value
                }
                rows.append(row)
    return pd.DataFrame(rows)

In [18]:
dataplot = flatten_dict(valores)
dataplot = dataplot[dataplot['campo'].str.contains('media')]
dataplot = dataplot\
.groupby(by=['integralizacao', 'campo'])\
.agg([np.mean])\
.reset_index()
dataplot.columns = dataplot.columns.get_level_values(0)
dataplot['valor'] = dataplot['valor'].apply(lambda x: 1 if x >= 6.0 else 0)
dataplot.to_csv('medias_por_integralizacao.csv')

### Regressão Linear

In [19]:
def regressao(df, idx):
    rows = []
    tmp = df[df['integralizacao']==idx].iloc[:, 3:]
    for item in tmp.columns:
        for item2 in tmp.columns:
            if item != item2:
                tmp2 = tmp[[item, item2]].dropna()
                X = tmp2[item].values.reshape(-1, 1)
                y = tmp2[item2].values.reshape(-1, 1)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
                regressor = LinearRegression()
                regressor.fit(X_train, y_train)
                y_pred = regressor.predict(X_test)
                row = { 'mse' : metrics.mean_absolute_error(y_test, y_pred),
                       'a' : item,
                       'b' : item2 }
                rows.append(row)
    tmp_df = pd.DataFrame(rows).sort_values(by='mse')
    tmp_df['a'] = tmp_df['a'].combine(tmp_df['b'], lambda x,y: x + ',' + y)
    tmp_df = tmp_df[['mse', 'a']]
    tmp_df = tmp_df[tmp_df['mse'] != 0]
    return tmp_df.head(5)

In [20]:
result = regressao(df, 0)
result['integralizacao'] = 0
for item in set(df['integralizacao'].values):
    if item != 0:
        tmp = regressao(df, item)
        tmp['integralizacao'] = item
        result = result.append(tmp)

In [21]:
result['mse'] = result['mse'].round(2)
# Essa é a tabela resultante da regressão linear
result.to_csv('regressao_linear.csv')

###

In [22]:
tmp_medidas = medidas\
.iloc[:, 2:]\
.reset_index()
tmp_medidas['level_0'] = tmp_medidas['level_0'].apply(lambda x: x.rstrip(','))
tmp_medidas.index = tmp_medidas['level_0']
tmp_medidas = tmp_medidas.drop(columns='level_0')
tmp_medidas = tmp_medidas.to_dict()

for key in tmp_medidas.keys():
    for key2 in tmp_medidas[key]:
        tmp_medidas[key][key2] = [round(float(x), 2) \
                              for x in tmp_medidas[key][key2].split(',')[0:3]]

def get_num(x):
    try:
        tmp = float(x)
        re.match('^(?=.)([+-]?([0-9]*)(\.([0-9]+))?)$', x)
        return round(tmp, 2)
    except:
        return -1
        
tmp_valores = valores.iloc[:, 2:].to_dict()
for key in tmp_valores.keys():
    for key2 in tmp_valores[key]:
        tmp_valores[key][key2] = sorted(list(set([x for x in [ get_num(x) for x \
                                          in tmp_valores[key][key2].split(',')]
                              if x != -1 ])))

pertinencias = {}
for key in tmp_valores.keys():
    pertinencias[key] = {}
    for key2 in tmp_valores[key]:
        pertinencias[key][key2] = None
        
for key in tmp_valores.keys():
    for key2 in tmp_valores[key]:
        try:
            arr = sorted(np.array(tmp_medidas[key][key2]))
        except:
            continue
        bom = \
        [arr[1]] + [arr[2]] + [arr[2]]
        medio = \
        [arr[0]] + [arr[1]] + [arr[2]]
        ruim = \
        [arr[0]] + [arr[0]] + [arr[1]]
        pertinencias[key][key2] = {
            'bom': fuzz.trimf(np.array(tmp_valores[key][key2]), 
                                             np.array(bom)),
            'medio': fuzz.trimf(np.array(tmp_valores[key][key2]), 
                                             np.array(medio)),
            'ruim': fuzz.trimf(np.array(tmp_valores[key][key2]), 
                                             np.array(ruim))
        }                  

/home/luana/miniconda3/envs/IA/lib/python3.7/site-packages/skfuzzy/membership/generatemf.py:441: RuntimeWarning: invalid value encountered in less
  idx = np.nonzero(np.logical_and(a < x, x < b))[0]
/home/luana/miniconda3/envs/IA/lib/python3.7/site-packages/skfuzzy/membership/generatemf.py:446: RuntimeWarning: invalid value encountered in less
  idx = np.nonzero(np.logical_and(b < x, x < c))[0]


In [23]:
classificacao = {
    "bom": fuzz.trimf(np.arange(0,10,1), [5, 10, 10]),
    "medio": fuzz.trimf(np.arange(0,10,1), [0, 5, 10]),
    "ruim": fuzz.trimf(np.arange(0,10,1), [0, 0, 5])
}

In [24]:
def get_user_data(df, idx):
    tmp = df[df['id']==idx]
    tmp = tmp\
    .iloc[:, 2:]
    tmp.index = tmp['integralizacao']
    tmp = tmp.drop(columns='integralizacao')
    return tmp.transpose().to_dict()

In [25]:
def activate(user, tmp_valores, pertinencias):
    ativacao = {}
    for key in tmp_valores.keys():
        ativacao[key] = {}
        for key2 in tmp_valores[key]:
            ativacao[key][key2] = {}
    
    for key in tmp_valores.keys():
        for key2 in tmp_valores[key]:
            try:
                value = user[key][key2]
                universe = tmp_valores[key][key2]
                for key3 in pertinencias[key][key2]:
                    ativacao[key][key2][key3] = \
                    fuzz.interp_membership(universe, 
                                           pertinencias[key][key2][key3],
                                           value)
            except:
                continue
    return ativacao

### Regras nebulosas

Uma regra nebulosa, ou um operador de implicação, são uma maneira de representar o conhecimento que expressam o que deve acontecer ou o que acontece quando certas condições são cumpridas, ou seja, é a criação de uma hipótese de implicação.

Para avaliar o desempenho ao longo dos períodos, definimos um conjunto de regras para cada período (faixa de integralização). As regras para cada período são:

- Período 0:
    - $(mediaProgramacao_{baixo} \land mediaNC_{baixo}) \to aluno_{mau}$
    - $(mediaProgramacao_{media} \land mediaNC_{media}) \to aluno_{medio}$
    - $(mediaProgramacao_{alto} \land mediaNC_{alto}) \to aluno_{bom}$  


- Período 1:    
    - $(reprovacoesNC_{alto}  \land reprovacoesMatematica_{alto}) \to aluno_{mau}$
    - $(reprovacoesNC_{medio} \land quantidadeTrancamento_{medio}) \to  aluno_{medio}$ 
    - $(reprovacoesNC_{baixo} \land reprovacoesMatematica_{baixo}) \to aluno_{bom}$


- Período 2:  
    - $(mediaNC_{baixo} \land mediaProgramacao_{baixo}) \to aluno_{mau}$ 
    - $(reprovacoesProgramacao_{medio} \land quantidadeTrancamento_{medio}) \to aluno_{medio}$
    - $(mediaNC_{alto} \land mediaProgramacao_{alto}) \to aluno_{bom}$ 


- Período 3:
    - $(mediaNC_{baixo} \land mediaProgramacao_{baixo}) \to aluno_{mau}$
    - $(mediaMatematica_{medio} \land quantidadeTrancamento_{medio}) \to aluno_{medio}$ 
    - $(mediaNC_{alto} \land mediaProgramacao_{alto}) \to aluno_{bom}$ 


- Período 4: 
    - $(mediaNE_{alto} \land mediaProgramacao_{alto}) \to aluno_{bom}$
    - $(mediaNE_{medio} \land quantidadeTrancamento_{medio}) \to aluno_{medio}$  
    - $(mediaNE_{baixo} \land mediaProgramacao_{baixo}) \to aluno_{mau}$  
    
    
- Período 5:   
    - $(mediaNE_{alto} \land mediaProgramacao_{alto}) \to aluno_{bom}$ 
    - $(reprovacoesMatematica_{medio} \land quantidadeTrancamento_{medio} )\to aluno_{medio}$  
    - $(mediaNE_{baixo} \land mediaProgramacao_{baixo}) \to aluno_{mau}$  


- Período 6:   
    - $(reprovacoesNC_{baixo} \land quantidadeTrancamentos_{baixo}) \to aluno_{bom}$  
    - $(reprovacoesNC_{medio} \land quantidadeTrancamentos_{medio}) \to aluno_{medio}$  
    - $(reprovacoesNC_{alto} \land quantidadeTrancamentos_{alto}) \to aluno_{mau}$  


- Período 7:    
    - $(reprovacoesNE_{baixo} \land quantidadeTrancamentos_{baixo}) \to aluno_{bom}$  
    - $(reprovacoesNE_{medio} \land quantidadeTrancamentos_{medio}) \to aluno_{medio}$  
    - $(reprovacoesNE_{alto} \land quantidadeTrancamentos_{alto}) \to aluno_{mau}$  


- Período 8:
    - $(reprovacoesMatematica_{baixo} \land quantidadeTrancamentos_{baixo})\to aluno_{bom}$  
    - $(reprovacoesMatematica_{medio} \land quantidadeTrancamentos_{medio})\to aluno_{medio}$  
    - $(reprovacoesMatematica_{alto} \land quantidadeTrancamentos_{alto})\to aluno_{mau}$


- Período 9:    
    - $(reprovacoesNE_{baixo} \land reprovacoesProgramacao_{baixo}) \to aluno_{bom}$  
    - $(reprovacoesNE_{medio} \land reprovacoesProgramacao_{medio}) \to aluno_{medio}$  
    - $(reprovacoesNE_{alto} \land reprovacoesProgramacao_{alto}) \to aluno_{mau}$ 


- Período 10: 
    - $(reprovacoesNC_{baixo} \land reprovacoesProgramacao_{baixo}) \to aluno_{bom}$  
    - $(reprovacoesNC_{medio} \land reprovacoesProgramacao_{medio}) \to aluno_{medio}$  
    - $(alto reprovacoesNC_{alto} \land reprovacoesProgramacao_{alto}) \to aluno_{mau}$   




Uma vez escolhido as regras, poderemos então realizar uma classificação de um dado aluno.

### Operadores e implicações

Dado uma observação, ou seja, um conjunto de valores para as variáveis do universo, queremos como resultado um valor de saída (classificação do aluno).

Para cada variável, será olhado o grau de pertencimento do dado valor para cada uma das possíveis faixas de valores da variável em questão. Isso será relizado utilizado a ativacoes() definida acima.

Dado o conjunto de regras nebulosas, iremos determinar a ativação das faixas de valores da variável de saída.

O conjunto de regras será divido em 3 partes, onde cada uma irá representar, respectivamente, a classificação de um bom, médio e mal aluno. Para cada conjunto de regra será calculado o grau máximo e mínimo de pertinência do conjunto utilizando conectivos lógicos.

Para os conectivos de conjunção, disjunção e negação das variáveis lógicas nebulosas, temos que:
- Conjunção: $A \vee B = Max(A, B)$
- Disjunção: $A \land B = Min(A, B)$
- Negação: $\neg A = 1 - A$

Para regras de implicação, ou seja, $se A \to B$, nos interessa obsersar se A e B são verdadeiros, ou seja, $A \land B$.

O próximo passo será a realização da combinação de todas as saídas em um único conjunto difuso, algo semelhante ao processo de união e intersecção.

Uma vez que já se sabe quais regras devem ser ativadas, será realizado a 'defuzzyficação', que consiste na obtenção de um valor numérico dentro da faixa estipulada pela lógica fuzzy, ou seja, iremos decifrar o significado de uma ação vaga. Para essa etapa será utilizada a função centróide, que calcula a média média aritmética pontderada pelas pertinências de cada elemento do cojunto difuso.

Todo esse procedimento descrito acima, será realizado para cada período relativo ao aluno.

In [26]:
ativacoes = activate(get_user_data(df, 30), 
                     tmp_valores, 
                     pertinencias)

def classify(ativacoes):
    result = {}
    try:
        # 0
        # baixo media_programacao OU baixo media_NC -> mau aluno
        # media media_programacao E media media_NC -> medio aluno
        # alto media_programacao E alto media_NC -> bom aluno
        a = ativacoes[0]['media_programacao']['ruim']
        b = ativacoes[0]['media_NC']['ruim']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmax(a,b), c)
        a = ativacoes[0]['media_programacao']['medio']
        b = ativacoes[0]['media_NC']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[0]['media_programacao']['bom']
        b = ativacoes[0]['media_NC']['bom']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[0] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass

    try:
        # 1
        # alto reprovacoes_NC OU alto reprovacoes_matematica -> mau aluno
        # medio reprovacoes_NC E medio quantidade_trancamento -> medio aluno
        # baixo reprovacoes_NC E baixo reprovacoes_matematica -> bom aluno
        a = ativacoes[1]['reprovacoes_NC']['bom']
        b = ativacoes[1]['reprovacoes_matematica']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmax(a,b), c)
        a = ativacoes[1]['reprovacoes_NC']['medio']
        b = ativacoes[1]['reprovacoes_matematica']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[1]['reprovacoes_NC']['ruim']
        b = ativacoes[1]['reprovacoes_matematica']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[1] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass

    try:
        # 2
        # baixo media_NC e baixo media_programacao -> mau aluno
        # medio reprovacoes_programacao e medio quantidade_trancamento -> medio aluno
        # alto media_NC e alto media_programacao -> bom aluno
        a = ativacoes[2]['media_NC']['ruim']
        b = ativacoes[2]['media_programacao']['ruim']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[2]['reprovacoes_programacao']['medio']
        b = ativacoes[2]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[2]['media_NC']['bom']
        b = ativacoes[2]['media_programacao']['bom']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[2] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    try:
        # 3
        # baixo media_NC e baixo media_programacao -> mau aluno
        # medio media_matematica e medio quantidade_trancamento -> medio aluno
        # alto media_NC e alto media_programacao -> bom aluno
        a = ativacoes[3]['media_NC']['ruim']
        b = ativacoes[3]['media_programacao']['ruim']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[3]['media_matematica']['medio']
        b = ativacoes[3]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[3]['media_NC']['bom']
        b = ativacoes[3]['media_programacao']['bom']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[3] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    try:
        # 4
        # alto media_NE e alto media_programacao -> bom aluno
        # medio media_NE e medio quantidade_trancamento -> medio aluno
        # baixo media_NE e baixo media_programacao -> mau aluno
        a = ativacoes[4]['media_NE']['ruim']
        b = ativacoes[4]['media_programacao']['ruim']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[4]['media_NE']['medio']
        b = ativacoes[4]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[4]['media_NE']['bom']
        b = ativacoes[4]['media_programacao']['bom']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[4] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    try:
        # 5
        # alto media_NE e alto media_programacao -> bom aluno
        # medio reprovacoes_matematica e medio quantidade_trancamento -> medio aluno
        # baixo media_NE e baixo media_programacao -> mau aluno
        a = ativacoes[5]['media_NE']['ruim']
        b = ativacoes[5]['media_programacao']['ruim']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[5]['reprovacoes_matematica']['medio']
        b = ativacoes[5]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[5]['media_NE']['bom']
        b = ativacoes[5]['media_programacao']['bom']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[5] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    try:
        # 6
        # baixo reprovacoes_NC e baixo quantidade_trancamentos -> bom aluno
        # medio reprovacoes_NC e medio quantidade_trancamentos -> medio aluno
        # alto reprovacoes_NC e alto quantidade_trancamentos -> mau aluno
        a = ativacoes[6]['reprovacoes_NC']['bom']
        b = ativacoes[6]['quantidade_trancamentos']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[6]['reprovacoes_NC']['medio']
        b = ativacoes[6]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[6]['reprovacoes_NC']['ruim']
        b = ativacoes[6]['quantidade_trancamentos']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[6] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    
    try:
        # 7
        # baixo reprovacoes_NE e baixo quantidade_trancamentos -> bom aluno
        # medio reprovacoes_NE e medio quantidade_trancamentos -> medio aluno
        # alto reprovacoes_NE e alto quantidade_trancamentos -> mau aluno
        a = ativacoes[7]['reprovacoes_NE']['bom']
        b = ativacoes[7]['quantidade_trancamentos']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[7]['reprovacoes_NE']['medio']
        b = ativacoes[7]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[7]['reprovacoes_NE']['ruim']
        b = ativacoes[7]['quantidade_trancamentos']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[7] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass

    try:
        # 8
        # baixo reprovacoes_matematica e baixo quantidade_trancamentos -> bom aluno
        # medio reprovacoes_matematica e medio quantidade_trancamentos -> medio aluno
        # alto reprovacoes_matematica e alto quantidade_trancamentos -> mau aluno
        a = ativacoes[8]['reprovacoes_matematica']['bom']
        b = ativacoes[8]['quantidade_trancamentos']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[8]['reprovacoes_matematica']['medio']
        b = ativacoes[8]['quantidade_trancamentos']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[8]['reprovacoes_matematica']['ruim']
        b = ativacoes[8]['quantidade_trancamentos']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[8] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    
    try:
        # 9
        # baixo reprovacoes_NE e baixo reprovacoes_programacao -> bom aluno
        # medio reprovacoes_NE e medio reprovacoes_programacao -> medio aluno
        # alto reprovacoes_NE e alto reprovacoes_programacao -> mau aluno
        a = ativacoes[9]['reprovacoes_NE']['bom']
        b = ativacoes[9]['reprovacoes_programacao']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[9]['reprovacoes_NE']['medio']
        b = ativacoes[9]['reprovacoes_programacao']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[9]['reprovacoes_NE']['ruim']
        b = ativacoes[9]['reprovacoes_programacao']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[9] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass

    try:
        # 10
        # baixo reprovacoes_NC e baixo reprovacoes_programacao -> bom aluno
        # medio reprovacoes_NC e medio reprovacoes_programacao -> medio aluno
        # alto reprovacoes_NC e alto reprovacoes_programacao -> mau aluno
        a = ativacoes[10]['reprovacoes_NC']['bom']
        b = ativacoes[10]['reprovacoes_programacao']['bom']
        c = classificacao['ruim']
        ruim = np.fmin(np.fmin(a,b), c)
        a = ativacoes[10]['reprovacoes_NC']['medio']
        b = ativacoes[10]['reprovacoes_programacao']['medio']
        c = classificacao['medio']
        medio = np.fmin(np.fmin(a,b), c)
        a = ativacoes[10]['reprovacoes_NC']['ruim']
        b = ativacoes[10]['reprovacoes_programacao']['ruim']
        c = classificacao['bom']
        bom = np.fmin(np.fmin(a,b), c)

        agg = np.fmax(ruim,
                         np.fmax(medio, bom))
        result[10] = fuzz.defuzz(np.arange(0,10,1), agg, 'centroid')
    except:
        pass
    return result

classify(ativacoes)

KeyError: 'integralizacao'

### Classificação

Após feito a classificação de um dado aluno ao longo de todos os seus períodos, podemos então visualizar seu gráfico de desempenho ao longo do tempo.

In [ ]:
from matplotlib.ticker import FuncFormatter
formatter = FuncFormatter(lambda x_val, tick_pos: "${{{0}}}0\%$".format(int(x_val)))

def plot_classification(result):
    g = sns.lineplot(x=0,
                        y=1,
                        data=pd.DataFrame(sorted(result.items(), key=lambda x: x[0])))
    g.xaxis.set_major_formatter(formatter)
    g.set_title('Desempenho do aluno')
    g.set(ylabel='Desempenho')
    g.set(xlabel='Porcentagem de conclusão')

plot_classification(classify(ativacoes))